## Import libraries

In [ ]:
from io import BytesIO
import os

from craft_ai_sdk import CraftAiSdk
from craft_ai_sdk.io import Input, Output, DeploymentInputMapping, DeploymentOutputMapping
from dotenv import load_dotenv, find_dotenv
import numpy as np
import pandas as pd
import requests
from sklearn import datasets

# from iris_train import train_iris_inputs, train_iris_outputs
# from iris_predict import predict_inputs, predict_outputs

In [ ]:
load_dotenv()

## SDK instantiation

In [ ]:
sdk = CraftAiSdk(
    environment_url=os.environ.get("CRAFT_AI_ENVIRONMENT_URL"),
    sdk_token=os.environ.get("CRAFT_AI_ACCESS_TOKEN"),
    verbose_log=True
)

## Upload dataset to Data Store

In [ ]:
iris = datasets.load_iris(as_frame=True)
iris_df = pd.concat([iris.data, iris.target], axis=1)

file_buffer = BytesIO(iris_df.to_parquet())
sdk.upload_data_store_object(filepath_or_buffer=file_buffer, object_path_in_datastore="get_started/dataset/iris.parquet")

In [ ]:
sdk.list_data_store_objects()

## Create Train Step

### Declare inputs and outputs

In [ ]:
train_val_prop_input = Input(name="train_val_prop", data_type="number")
model_parameters_input = Input(name="model_parameters", data_type="json")
dataset_input = Input(name="dataset_path", data_type="string")
model_output_path = Input(name="model_output_path", data_type="string")
train_iris_inputs = [
    train_val_prop_input,
    model_parameters_input,
    dataset_input,
    model_output_path,
]

metrics_file_output = Output(name="metrics", data_type="json")
train_iris_outputs = [metrics_file_output]

### Create the step and pipeline

In [ ]:
step = sdk.create_step(
    # The path must be relative to the root of the repo
    function_path="src/part-4-iris_train.py",
    function_name="train",
    inputs=train_iris_inputs,
    outputs=train_iris_outputs,
    step_name="part-4-iris-train-step",
)

In [ ]:
pipeline = sdk.create_pipeline(
    pipeline_name="part-4-iris-train-pipeline",
    step_name="part-4-iris-train-step",
)

In [ ]:
output_mappings = [
    DeploymentOutputMapping(
        step_output_name='metrics',
        endpoint_output_name='metrics')
]

endpoint = sdk.create_endpoint(
    pipeline_name="part-4-iris-train-pipeline",
    endpoint_name="part-4-iris-train-endpoint",
    outputs_mapping=output_mappings
)

print(endpoint)

In [ ]:
endpoint_url = sdk.base_environment_url  + "/endpoints/" + endpoint["name"]
inputs = {
		"dataset_path": "get_started/dataset/iris.parquet",
    "train_val_prop": 0.8,
    "model_parameters": {"n_neighbors": 10},
    "model_output_path": "get_started/models/test_model.joblib"
}
endpoint_token = endpoint["endpoint_token"]

request = requests.post(endpoint_url, headers={"Authorization": f"EndpointToken {endpoint_token}"}, json=inputs)
request.json()

In [ ]:
endpoint_url = sdk.base_environment_url  + "/endpoints/" + endpoint["name"]
inputs = {
    "dataset_path": "get_started/dataset/iris.parquet",
    "train_val_prop": 0.8,
    "model_parameters": {"n_neighbors": 1},
    "model_output_path": "get_started/models/test_model_2.joblib"
}
endpoint_token = endpoint["endpoint_token"]

request = requests.post(endpoint_url, headers={"Authorization": f"EndpointToken {endpoint_token}"}, json=inputs)
request.json()

In [ ]:
sdk.list_data_store_objects()

In [ ]:
model_path_input = Input(name="model_path", data_type="string")
input_data_input = Input(name="input_data", data_type="json")
predict_inputs = [
    model_path_input,
    input_data_input,
]

predictions_output = Output(name="predictions", data_type="json")
predict_outputs = [predictions_output]

In [ ]:
step = sdk.create_step(
    step_name="part-4-iris-predict-step",
    function_path="src/part-4-iris_predict.py",
    function_name="predict",
    inputs=predict_inputs, 
    outputs=predict_outputs,
    container_config={
        "requirements_path": "requirements.txt"
    },
)

In [ ]:
pipeline = sdk.create_pipeline(
    step_name="part-4-iris-predict-step",
    pipeline_name="part-4-iris-predict-pipeline"
)

In [ ]:
input_mappings = [
    DeploymentInputMapping(
        step_input_name="model_path",
        constant_value="get_started/models/test_model.joblib")
]

output_mappings = [
    DeploymentOutputMapping(
        step_output_name="predictions",
        endpoint_output_name="iris_species")
]

In [ ]:
endpoint = sdk.create_endpoint(
    pipeline_name="part-4-iris-predict-pipeline",
    endpoint_name="part-4-iris-predict-endpoint",
    inputs_mapping=input_mappings,
    outputs_mapping=output_mappings
)

In [ ]:
np.random.seed(0)
indices = np.random.permutation(150)
iris_X, iris_y = datasets.load_iris(return_X_y=True, as_frame=True)
iris_X_test = iris_X.loc[indices[90:120],:]
input_json_data = iris_X_test.to_dict(orient="index")

In [ ]:
endpoint_url = sdk.base_environment_url  + "/endpoints/" + endpoint["name"]


inputs = {"input_data": input_json_data}
endpoint_token = endpoint["endpoint_token"]

request = requests.post(endpoint_url, headers={"Authorization": f"EndpointToken {endpoint_token}"}, json=inputs)
request.json()["results"]["iris_species"]